In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Activation,Conv2D,MaxPooling2D,Flatten,GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers, optimizers
from matplotlib import pyplot as plt
import shutil
import cv2
from PIL import Image

In [ ]:
os.chdir('./kaggle/input')
os.listdir()

In [ ]:
os.chdir('./train_images')
train_images = os.listdir()
os.chdir('./../test_images')
test_images = os.listdir()
os.chdir('./../')
os.listdir()

In [ ]:
train_csv = pd.read_csv('./train.csv')
test_csv  = pd.read_csv('./test.csv')

In [ ]:
train_csv['id_code'] = train_csv['id_code'].map(lambda x : x+'.png')
test_csv['id_code'] = test_csv['id_code'].map(lambda x : x+'.png')

In [ ]:
train_csv['diagnosis'] = train_csv['diagnosis'].apply(str)

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(train_csv.id_code,train_csv.diagnosis,test_size=0.2)

In [ ]:
def crop_image_from_gray(img,folder,tol=7,):
    if folder == 'train':
        path = f'./train_images/{label}'
        img = cv2.imread(path)
        if img.ndim == 2:
            mask = img > tol
            return img[np.ix_(mask.any(1), mask.any(0))]
        elif img.ndim == 3:
            gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            mask = gray_img > tol
            check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
            if (check_shape == 0):
                return img
            else:
                img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
                img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
                img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
                img = np.stack([img1, img2, img3], axis=-1)
                dim = (240,120)
                img = cv2.resize(img,dim,interpolation = cv2.INTER_AREA)
            return img
    elif folder == 'test':
        path = f'./test_images/{label}'
        img = cv2.imread(path)
        if img.ndim == 2:
            mask = img > tol
            return img[np.ix_(mask.any(1), mask.any(0))]
        elif img.ndim == 3:
            gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            mask = gray_img > tol
            check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
            if (check_shape == 0):
                return img
            else:
                img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
                img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
                img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
                img = np.stack([img1, img2, img3], axis=-1)
                dim = (240,120)
                img = cv2.resize(img,dim,interpolation = cv2.INTER_AREA)
            return img
        

      
def circle_crop(img,folder):
    
    img = crop_image_from_gray(img,folder)
  
    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img,folder)

    return img

In [ ]:
def img_to_green(img,folder):
  
    circle_crop(img,folder)
    dim = (224,224)
    image = cv2.imread(path)
    image = image[:,:,1]
    clahe = cv2.createCLAHE(clipLimit=30.0, tileGridSize=(8,8))
    image = cv2.medianBlur(image,3)
    image = cv2.equalizeHist(image)
    image = clahe.apply(image)
    image = cv2.resize(image,dim,interpolation = cv2.INTER_AREA)
    
    return image


In [ ]:
def preprocess_image(path,folder):
    img = cv2.imread(path)
    img = img_to_green(img,folder)
    cv2.imwrite(path,img)

In [ ]:
folder = 'train'
for label in train_images:
    path = f'./train_images/{label}'
    preprocess_image(path,'train')

In [ ]:
for label in test_images:
    path = f'./test_images/{label}'
    preprocess_image(path,'test')

In [ ]:
generator = ImageDataGenerator(rescale=1.0/255.0,
                              validation_split=0.1,
                              horizontal_flip=True,
                              vertical_flip=True,
                              )
train_path = './train_images/'
train_generator = generator.flow_from_dataframe(
             dataframe = train_csv,
             directory = train_path,
             subset='training',
             target_size=(224,224),
             x_col = 'id_code',
             y_col = 'diagnosis',
             class_mode='categorical',
             shuffle=True,
             batch_size = 32)

In [ ]:
validation_generator = generator.flow_from_dataframe(dataframe=train_csv,
                                                    directory=train_path,
                                                    subset='validation',
                                                    target_size=(224,224),
                                                    x_col='id_code',
                                                    y_col='diagnosis',
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    batch_size=32)

In [ ]:
test_path = './test_images/'
test_generator = generator.flow_from_dataframe(dataframe=test_csv,
                                             directory=test_path,
                                             target_size=(224,224),
                                             x_col='id_code',
                                             y_col=None,
                                             class_mode=None,
                                             shuffle=False,
                                             batch_size=32)

In [ ]:
densenet = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
model = Sequential()
model.add(densenet)
model.add(GlobalAveragePooling2D)
model.add(Flatten())
model.add(Dropout(0.6))
model.add(Dense(units=256,activation='relu')
model.add(Dropout(0.5))
model.add(Dense(units=512,activation='relu'))
model.add(Dense(units=5,activation='softmax'))        

In [ ]:
model.summary()

In [ ]:
model.compile(tf.keras.optimizers.RMSprop(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [ ]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=12)

In [ ]:
model.evaluate_generator(generator=validation_generator)

In [ ]:
test_generator.reset()

pred=model.predict_generator(test_generator,verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
sub = pd.read_csv('./sample_submission.csv')
submission_csv = pd.DataFrame({'id_code': sub['id_code'], 'diagnosis': predicted_class_indices})

In [ ]:
submission_csv.shape

In [ ]:
submission_csv.to_csv('submission.csv',index=False)